# Interobserver Realiability test

### Creating labelling templates

In [ ]:
import os
import shutil
import random
import pandas as pd

CSV_PATH = "/lisc/data/scratch/becogbio/juarez/thesis/3_Viewpoint_classifier/viewpoint_labelling.csv"
SRC_IMG_DIR = "/lisc/data/scratch/becogbio/juarez/thesis/3_Viewpoint_classifier/frames_for_labelling"
DEST_DIR    = "/lisc/data/scratch/becogbio/juarez/thesis/3_Viewpoint_classifier/IOR_test/IOR_test_instances"
DEST_CSV    = os.path.join(DEST_DIR, "IOR_test_instances_metadata.csv")

RANDOM_SEED = 2025          
SAMPLE_SIZE = 100 # number of viewpoint instances to sample

def ensure_dest_dir(path: str) -> None:
    os.makedirs(path, exist_ok=True)

def main() -> None:
    random.seed(RANDOM_SEED)

    df = pd.read_csv(CSV_PATH)

    required_columns = {"annotation_id", "image", "label"}
    missing = required_columns - set(df.columns)
    if missing:
        raise ValueError(f"CSV is missing expected columns: {missing}")

    # Sample 100 rows for every unique viewpoint
    sampled_frames = []
    for label, subset in df.groupby("label"):
        if len(subset) < SAMPLE_SIZE:
            raise ValueError(
                f"Only {len(subset)} rows for viewpoint “{label}”, "
                f"but {SAMPLE_SIZE} requested."
            )
        sampled = subset.sample(n=SAMPLE_SIZE, random_state=RANDOM_SEED)
        sampled_frames.append(sampled)

    sampled_df = pd.concat(sampled_frames, ignore_index=True)

    # Copy the images
    ensure_dest_dir(DEST_DIR)
    for _, row in sampled_df.iterrows():
        src_file  = os.path.join(SRC_IMG_DIR, row["image"])
        dest_file = os.path.join(DEST_DIR,    row["image"])

        if not os.path.isfile(src_file):
            raise FileNotFoundError(f"Image not found: {src_file}")

        shutil.copy2(src_file, dest_file) # preserves timestamps & metadata

    # Build and save the metadata CSV
    out_df = (
    sampled_df[["image", "label"]] # keep viewpoint instead
    .rename(columns={"label": "annotations_annotator1"})
    .assign(annotations_lulu="") # empty column for Lulu
)
    out_df.to_csv(DEST_CSV, index=False)
    print(f"✓ Done! {len(out_df)} files copied to {DEST_DIR}")
    print(f"  Metadata written to {DEST_CSV}")

if __name__ == "__main__":
    main()

### Run actual IOR

* Raw agreement (%)
* Cohen’s Kappa
* Confusion matrix

In [ ]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score, confusion_matrix

csv_path = "/lisc/data/scratch/becogbio/juarez/thesis/3_Viewpoint_classifier/IOR_test/IOR_labels.csv"

# auto-detect delimiter
df = pd.read_csv(csv_path, sep=None, engine="python")
df.columns = [c.strip().lower().lstrip("\ufeff") for c in df.columns]  # handle BOM + spacing
df = df[["image", "annotations_annotator1", "annotations_annotator2"]].dropna()

# metrics
labels = ["front", "back", "left_side", "right_side"]
agreement = (df["annotations_annotator1"] == df["annotations_annotator2"]).mean() * 100
kappa = cohen_kappa_score(df["annotations_annotator1"], df["annotations_annotator2"])
cm = confusion_matrix(df["annotations_annotator1"], df["annotations_annotator2"], labels=labels)

# report
print(f"Rows used: {len(df)}")
print(f"Raw Agreement: {agreement:.2f}%")
print(f"Cohen's Kappa: {kappa:.3f}")
print("Confusion Matrix (rows = annotator1, cols = annotator2):")
print(pd.DataFrame(cm, index=labels, columns=labels))
